In [1]:
import sys
sys.path.append("/home/laba/golos/whisper_fine_tuning/dataset_creation/src/")

import numpy as np
# import matplotlib.pyplot as plt
import os
import pandas as pd
from tqdm import tqdm
import librosa
import glob

from utils import load_labels
from utils_model import load_model, predict_video
from utils_analysis import calculate_wer, clean_text_before_wer

In [51]:
labels = load_labels('results/filtered_labels_train.jsonl')
model_small = load_model('small')

In [3]:
model_finetune = load_model('small', model_path="experiments/small_bs_12_epoch_12/ckpt_epoch_best_epoch_iter_.pt")

In [52]:
prediction_finetune = predict_video(video_id = 3,
                                dataset_path = "dataset/",
                                model = model_finetune.module,
                                labels = labels,
                                pred_method='base')
calculate_wer(prediction_finetune)

In [53]:
# (0.19458135968521703, 0.14285714285714285)
# (0.19074436540423984, 0.14285714285714285)

# 166 - (0.26818501769276787, 0.2222222222222222)

prediction_finetune.wer.mean(), prediction_finetune.wer.median()

(0.04994955631400501, 0.02631578947368421)

In [47]:
prediction_small = predict_video(video_id = 3,
                                         dataset_path = "dataset/",
                                         model = model_small,
                                         labels = labels,
                                         pred_method='base')

calculate_wer(prediction_small)

In [12]:
prediction_small.wer.mean(), prediction_small.wer.median()

(0.259469253488408, 0.23076923076923078)

In [8]:
prediction_finetune

,label,prediction,clean_label,clean_prediction,wer
0,Утомилися від карантину Дарма Заплануйте втому...,втомились від карантину дармаааа запламуєте в ...,утомилися від карантину дарма заплануйте втому...,втомились від карантину дармаааа запламуєте в ...,0.384615
1,Слава Ісу Ви дивитесь програму Ґрати песик дуж...,слава ісу ви дивитесь програму ґрати песик дуж...,слава ісу ви дивитесь програму ґрати песик дуж...,слава ісу ви дивитесь програму ґрати песик дуж...,0.166667
2,НеКей тобто Настя Каменських записала карантин...,енкейт тобто настя каменських записала каранти...,некей тобто настя каменських записала карантин...,енкейт тобто настя каменських записала каранти...,1.187500
3,З точки зору бізнесу це дуже розумний хід Адже...,с точки зору бізнесу це дуже розумний хіт адже...,з точки зору бізнесу це дуже розумний хід адже...,с точки зору бізнесу це дуже розумний хіт адже...,0.062500
4,Вот ю гона ду вот ю гона ду Ін карантін моуд ...,what you gotta do what you gotta do in quaran...,вот ю гона ду вот ю гона ду ін карантін моуд а...,what you gotta do what you gotta do in quarant...,0.548387
...,...,...,...,...,...
126,Ага через пару років ми побачимо як Ґрета Тунб...,ага через пару років ми побачимо як ґрета тунб...,ага через пару років ми побачимо як ґрета тунб...,ага через пару років ми побачимо як ґрета тунб...,0.360000
127,миииии Не дивлячись на палання горіння та апок...,вииииииии не дивлячись на палання горіння та ...,миииии не дивлячись на палання горіння та апок...,вииииииии не дивлячись на палання горіння та а...,0.088235
128,Дякую Хардкісс Так подякувати гурту справді ва...,дякую хард кіс так подякувати гурту справді ва...,дякую хардкісс так подякувати гурту справді ва...,дякую хард кіс так подякувати гурту справді ва...,0.192308
129,Сподобалось відео Ставте лайк Не сподобалось С...,сподобалось відео ставте лайк не сподобалось с...,сподобалось відео ставте лайк не сподобалось с...,сподобалось відео ставте лайк не сподобалось с...,0.000000


In [24]:
prediction_small[prediction_small.wer>prediction_finetune.wer].shape

(90, 5)

In [25]:
(prediction_small.wer - prediction_finetune.wer).describe()

count    131.000000
mean       0.059494
std        0.122535
min       -0.500000
25%        0.000000
50%        0.058824
75%        0.121212
max        0.470588
Name: wer, dtype: float64

In [45]:
result = pd.concat([prediction_small, prediction_finetune.add_prefix('fine_tune_')], axis=1)

In [53]:
result[result.wer<result.fine_tune_wer].sort_values('fine_tune_wer', ascending=False).iloc[3]

label                         І поки ви в паніці кидаєте королю Таїланду в о...
prediction                     І поки ви у пані цекидаєте корлю Таїланду в о...
clean_label                   і поки ви в паніці кидаєте королю таїланду в о...
clean_prediction              і поки ви у пані цекидаєте корлю таїланду в ос...
wer                                                                     0.47619
fine_tune_label               І поки ви в паніці кидаєте королю Таїланду в о...
fine_tune_prediction          і поки ви у пані це кидаєте корлю та іланду в ...
fine_tune_clean_label         і поки ви в паніці кидаєте королю таїланду в о...
fine_tune_clean_prediction    і поки ви у пані це кидаєте корлю та іланду в ...
fine_tune_wer                                                           0.52381
Name: 36, dtype: object

In [52]:
result[result.wer<result.fine_tune_wer].sort_values('fine_tune_wer', ascending=False).iloc[3][['label', 'prediction', 'fine_tune_prediction']].values

array(['І поки ви в паніці кидаєте королю Таїланду в особисті повідомлення новий кліп Дуа Ліпи де вона співає що краще все ж таки вдома залишатись I would ve stayed at home Cause I was doing better alone Я закликаю вас не хвилюватись ',
       ' І поки ви у пані цекидаєте корлю Таїланду в особисі повідомлення новий кліп «Дуаліпи», де вона співає, що краще все ж таки дома залишатись. «А я закликаю вас не хвилюватись».',
       'і поки ви у пані це кидаєте корлю та іланду в особисі повідомлення новий кліп дуаліпи де вона співає що краще все ж таки дома залишатись я закликаю вас не хвилюватись '],
      dtype=object)

In [16]:
# df = pd.read_csv('results/labels_baseline_prediction.csv')
# df_eval = pd.read_csv('results/labels_eval_baseline_prediction.csv')

# df = pd.concat([df, df_eval])
# print(df.shape)
# df.head()

In [58]:
prediction_finetune.sort_values('wer', ascending=False).iloc[1][['label', 'prediction', 'wer']].values

array(['Щедрий вечір добрий вечір Добрим людям на здоров’я Бога тобі того Детя тулько носит І навіть тональність та ж сама Паскудно від того шо хтось називає себе',
       ' ще три вечора добре вечора добре людям на здоров’я о го тобі того дитя тулько носит і навіть тональність та ж сама паскудно від того що хтось називає себе',
       0.37037037037037035], dtype=object)

In [54]:
prediction_finetune.sort_values('wer', ascending=False).head(10)

,label,prediction,clean_label,clean_prediction,wer
39,Гадаю все буде добре Вау ти в’яжеш светр А чо...,вони мається все добре,гадаю все буде добре вау ти вяжеш светр а чого...,вони мається все добре,0.960000
29,Щедрий вечір добрий вечір Добрим людям на здор...,ще три вечора добре вечора добре людям на здо...,щедрий вечір добрий вечір добрим людям на здор...,ще три вечора добре вечора добре людям на здор...,0.370370
52,І і і і Від імені усієї дельфінячої раси заяв...,вєди мені усієї дельфінячої раси заявляю що сн...,і і і і від імені усієї дельфінячої раси заявл...,вєди мені усієї дельфінячої раси заявляю що сн...,0.343750
71,завжди обирає втопити педаль газу на слизькій ...,завжди обирає втопити педаль газу на слизькій ...,завжди обирає втопити педаль газу на слизькій ...,завжди обирає втопити педаль газу на слизькій ...,0.260870
15,Днями Путін через силу проводить зустріч у Кре...,днями путін через силу проводить зустріч у кре...,днями путін через силу проводить зустріч у кре...,днями путін через силу проводить зустріч у кре...,0.222222
110,Ааааа Тобто український відповідник до квесту ...,тобто український відповідник до квесту фантас...,ааааа тобто український відповідник до квесту ...,тобто український відповідник до квесту фантас...,0.222222
62,Мене погодились взяти в автівку патрульних Біж...,мене погодились взяти в автівку патрульних біж...,мене погодились взяти в автівку патрульних біж...,мене погодились взяти в автівку патрульних біж...,0.192308
55,Уу йю Ха Ви дивитесь коротку нейтральну переби...,клюв і хаааааааа ви дивитесь коротку нейтраль...,уу йю ха ви дивитесь коротку нейтральну переби...,клюв і хаааааааа ви дивитесь коротку нейтральн...,0.176471
16,не дуже Так мріяти не заборониш Але давайте тв...,не дуже так мріяти не заборониш але давайте тв...,не дуже так мріяти не заборониш але давайте тв...,не дуже так мріяти не заборониш але давайте тв...,0.161290
91,із російськими військами на фоні зростаючого с...,і з російськими військами на фоні зростаючого ...,із російськими військами на фоні зростаючого с...,і з російськими військами на фоні зростаючого ...,0.121212


In [21]:
whisper_large_predictions = pd.read_csv("results/predictions.csv").dropna()

In [22]:
calculate_wer(whisper_large_predictions)

In [57]:
path_ = "dataset/toronto_3/toronto_3_39.wav"

whisper_large_predictions[whisper_large_predictions["wav_path"] == path_][['label', 'prediction', 'wer']].values

array([[' Гадаю все буде добре Вау ти в’яжеш светр А чого такий величезний Це для моєї коханої машини Я вже зв’язав їй шарф на вихлопну трубу ',
        " Все буде добре. Моооу, ти в'яжеш свет! А чого такий величезний? Ну, це для моєї коханої машини. Я вже зв'язав її шарф на вихлопну трубу.",
        0.2]], dtype=object)

In [29]:
len("З днем народження Фу фу вірою і правдою відслужив державі 17 років Пуделі одні з найрозумніших собак".split(" ")), len(" Хеп, фейбл, фуе, ту, ю, а че бау, ля, фу. Фу-фу, вірою і правдою, відслужив державі 17 років. Пуделі – одні з найрозумніших собак,".split(" "))

(17, 25)

In [60]:
from collections import Counter

combined_text = ''.join(whisper_large_predictions['label'].astype(str))


character_counts = Counter(combined_text)

print(character_counts)

Counter({' ': 432113, 'о': 203420, 'а': 193438, 'н': 144385, 'и': 137246, 'і': 124388, 'е': 120589, 'т': 120573, 'в': 107744, 'р': 106500, 'с': 88696, 'к': 86960, 'у': 79269, 'л': 77331, 'д': 69424, 'м': 65602, 'п': 60048, 'з': 45405, 'я': 44817, 'ь': 42008, 'б': 35452, 'г': 31639, 'й': 28941, 'ч': 25922, 'ж': 21704, 'ц': 20857, 'х': 20271, 'ю': 19367, 'ш': 18368, 'є': 14483, 'ї': 14221, 'щ': 12407, 'А': 7686, 'Т': 7236, 'ф': 6098, 'В': 5986, 'П': 5798, 'Н': 5543, 'М': 4766, 'С': 4625, 'І': 4410, 'Д': 4290, 'К': 3930, 'З': 3819, 'Б': 3512, 'У': 3459, 'О': 3312, 'Я': 3123, '’': 2452, 'Ц': 2187, 'Р': 2182, 'Г': 2074, '0': 1736, 'Л': 1671, 'Х': 1444, '1': 1424, 'Ч': 1404, 'Щ': 1178, '2': 1164, 'e': 1033, 'Ф': 981, 'Ш': 963, 'o': 809, 'a': 775, 'Е': 727, 'Є': 671, 't': 669, 'i': 656, 'n': 607, '5': 571, 's': 525, 'r': 479, 'Ю': 452, '9': 444, 'u': 405, '3': 400, '4': 386, 'h': 381, 'l': 361, 'Ж': 346, 'm': 309, 'd': 305, '6': 304, '7': 296, '8': 292, 'y': 273, 'g': 238, 'Й': 230, 'Ґ': 221,